In [ ]:
"""
https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
"""

In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### Get the correct device

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [16]:
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))]
    )

trainset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

Files already downloaded and verified
Files already downloaded and verified


In [17]:
def imshow(img):
    img = img/2 + 0.5   #unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1,2,0)))
    plt.show()

# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

print(images.shape)

#show images
#imshow(torchvision.utils.make_grid(images))
#print labels
#print(" ".join("%5s" % classes[labels[j]] for j in range(4)))

torch.Size([4, 3, 32, 32])


### Define a CNN

In [18]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
net.to(device)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

### Define a Loss function and optimizer

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)


### Train the network

In [20]:
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs, data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the gradients
        optimizer.zero_grad()

        # forward -> backward -> optimizer
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        #print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:       #print every 2000 mini-batches
            print("[%d, %5d] loss: %.3f" % (epoch+1, i+1, running_loss/2000))
            running_loss=0.0

print("Finished Training")

[1,  2000] loss: 2.111
[1,  4000] loss: 1.985
[1,  6000] loss: 1.978
[1,  8000] loss: 1.948
[1, 10000] loss: 1.971
[1, 12000] loss: 1.942
[2,  2000] loss: 1.982
[2,  4000] loss: 1.962
[2,  6000] loss: 1.961
[2,  8000] loss: 2.020
[2, 10000] loss: 2.012
[2, 12000] loss: 1.990
Finished Training


### Save the trained model

In [21]:
PATH = "./cifar_net.pth"
torch.save(net.state_dict(), PATH)

### Test the network on the test data

In [22]:
net = Net()
net.to(device)
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [23]:
correct = 0
total = 0

with torch.no_grad():
   for data in testloader:
       images, labels = data[0].to(device), data[1].to(device)
       outputs = net(images)
       _, predicted = torch.max(outputs.data, 1)
       total += labels.size(0)
       correct += (predicted == labels).sum().item()

print("Accuracy on the network on the 1000 test images: {} %".format(100*correct/total))

Accuracy on the network on the 1000 test images: 23.72 %


### Check which classes it predicted correctly

In [24]:
class_correct = list(0 for i in range(10))
class_total = list(0 for i in range(10))

with torch.no_grad():
    for data in testloader:
        images,labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        #print(c)
        #print(labels)
        for i in range(4):
            true_label = labels[i]
            class_correct[true_label] += c[i].item()
            class_total[true_label] += 1

for i in range(10):
    print("Accuracy of {}: {} %".format(classes[i], 100*class_correct[i] / class_total[i]))

Accuracy of plane: 21.0 %
Accuracy of car: 76.1 %
Accuracy of bird: 5.9 %
Accuracy of cat: 45.8 %
Accuracy of deer: 1.1 %
Accuracy of dog: 1.2 %
Accuracy of frog: 46.0 %
Accuracy of horse: 15.6 %
Accuracy of ship: 7.2 %
Accuracy of truck: 17.3 %
